In [15]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [16]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/svm-classifier.csv').drop('Unnamed: 0', axis=1)

In [17]:
### Season to test results

N = 2021

In [18]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(['season', 'round', 'podium'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [19]:
def score_classification(model):
    predictions = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium'], axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('proba_1', ascending=False, inplace=True)
        prediction_df.reset_index(inplace = True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)


        predictions += list(prediction_df['actual'].values)
        # print(prediction_df['actual'].values)
        # correct_predictions += precision_score(prediction_df.actual, prediction_df.predicted)

    # return correct_predictions / df[df.season == N]['round'].nunique()
    return predictions

In [20]:
params = parameters.head(1)
params

,model,gamma,C,kernel,score
0,svm_classifier,0.003793,0.020691,sigmoid,0.666667


In [21]:
gamma = params.gamma.values[0]
c = params.C.values[0]
kernel = params.kernel.values[0]

model_params = (gamma, c, kernel)
model = svm.SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions = score_classification(model)

In [24]:
merged = merged[['season', 'round', 'driver', 'starting_grid', 'podium']].query('season == @N')

merged['predicted'] = predictions

In [26]:
merged.query('predicted == 1')

,season,round,driver,starting_grid,podium,predicted
2761,2021,1,max_verstappen,1,2,1
2782,2021,2,lando_norris,7,3,1
2800,2021,3,max_verstappen,3,2,1
2819,2021,4,lewis_hamilton,1,1,1
2839,2021,5,max_verstappen,2,1,1
2861,2021,6,charles_leclerc,1,4,1
2876,2021,7,max_verstappen,1,1,1
2894,2021,8,max_verstappen,1,1,1
2914,2021,9,max_verstappen,1,1,1
2935,2021,10,charles_leclerc,4,2,1
